Como primera baseline predecimos para cada contador y cada día el mismo consumo de agua que tuvo ese mismo contador ese mismo día del año pero el año pasado.

Vamos a utilizar como test la última semana de Enero

In [105]:
import pandas as pd

data = pd.read_csv('../data/enriquecidos-003.csv', index_col=False)
data = data.drop('Unnamed: 0',axis=1)
data.head()

,DATE,ID,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,WEEK_DAY,...,YEAR_DAY,sin_WEEK_DAY,cos_WEEK_DAY,sin_YEAR_DAY,cos_YEAR_DAY,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,PRECIPITATIONS
0,2019-02-01,0,7963359,0.0,243,0.0,2019.0,2.0,1.0,4,...,32,-0.433884,-0.900969,0.523416,0.852078,16.5,17.6,18.7,7.1,0.0
1,2019-02-01,1,139882,0.0,8,0.0,2019.0,2.0,1.0,4,...,32,-0.433884,-0.900969,0.523416,0.852078,16.5,17.6,18.7,7.1,0.0
2,2019-02-01,2,190824,0.0,0,0.0,2019.0,2.0,1.0,4,...,32,-0.433884,-0.900969,0.523416,0.852078,16.5,17.6,18.7,7.1,0.0
3,2019-02-01,3,3611828,0.0,492,0.0,2019.0,2.0,1.0,4,...,32,-0.433884,-0.900969,0.523416,0.852078,16.5,17.6,18.7,7.1,0.0
4,2019-02-01,4,2723035,0.0,247,56.0,2019.0,2.0,1.0,4,...,32,-0.433884,-0.900969,0.523416,0.852078,16.5,17.6,18.7,7.1,0.0


In [106]:
len(data['ID'].unique())

2747

In [107]:
data[data['DATE']]

DATE
2019-02-01    2248
2019-02-02    2247
2019-02-03    2245
2019-02-04    2252
2019-02-05    2261
              ... 
2020-01-27    2596
2020-01-28    2598
2020-01-29    2607
2020-01-30    2618
2020-01-31    2632
Name: ID, Length: 365, dtype: int64

In [102]:
data['ID'].unique()[-47:]

array([2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710,
       2711, 2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721,
       2722, 2723, 2724, 2725, 2727, 2728, 2729, 2730, 2731, 2732, 2733,
       2734, 2735, 2736, 2737, 2739, 2742, 2743, 2744, 2745, 2746, 2747,
       2748, 2749, 2756], dtype=int64)

Contadores que no están:

- 2726
- 2738
- 2740
- 2741
- 2750
- 2751
- 2752
- 2753
- 2754
- 2755

In [59]:
def num_digitos_parte_entera(n):
    if n == 0:
        return 1
    i=0
    while n > 0:
        n//=10
        i+=1
    return i

In [75]:
january_last_week = ["2020-01-25","2020-01-26","2020-01-27","2020-01-28","2020-01-29",
                                                       "2020-01-30","2020-01-31"]
aux = [i in january_last_week for i in data['DATE'].values]
real = data[pd.Series(aux)]
real['DELTA'] = [j+(i/10**(num_digitos_parte_entera(i))) for (j,i) in zip(real['DELTAINTEGER'].values, real['DELTATHOUSANDTH'].values)]
real = real[['DATE','ID','DELTA']]
print(real.shape)
real

(18280, 3)


C:\Users\sergi\AppData\Local\Temp\ipykernel_18520\4101534024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['DELTA'] = [j+(i/10**(num_digitos_parte_entera(i))) for (j,i) in zip(real['DELTAINTEGER'].values, real['DELTATHOUSANDTH'].values)]


,DATE,ID,DELTA
872557,2020-01-25,0,390.000
872558,2020-01-25,1,0.132
872559,2020-01-25,2,30.000
872560,2020-01-25,3,639.000
872561,2020-01-25,4,266.514
...,...,...,...
890832,2020-01-31,2734,0.000
890833,2020-01-31,2735,69.000
890834,2020-01-31,2736,52.000
890835,2020-01-31,2737,0.000


In [79]:
len(real['ID'].value_counts())

2671

In [78]:
real.pivot(index='ID', columns='DATE', values='DELTA')

DATE,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
ID,,,,,,,
0,390.000,304.000,213.00,232.00,403.000,425.0,255.000
1,0.132,0.297,4.00,1.00,0.198,10.0,28.495
2,30.000,32.000,39.00,40.00,45.000,27.0,59.000
3,639.000,350.000,289.00,380.00,404.000,339.0,391.000
4,266.514,280.198,260.36,299.44,420.100,417.3,252.198
...,...,...,...,...,...,...,...
2734,NaN,NaN,NaN,NaN,NaN,NaN,0.000
2735,NaN,NaN,NaN,NaN,NaN,NaN,69.000
2736,NaN,NaN,NaN,NaN,NaN,NaN,52.000


In [42]:
from datetime import datetime

def str2date2(string):
    return datetime.strptime(string, '%Y-%m-%d')

In [32]:
def mse(real, prediction):
    

IndentationError: expected an indented block (3082681418.py, line 2)

In [33]:
data.shape

(890837, 23)

In [44]:
type(data['DATE'][0])

str

(18280, 23)

In [36]:
real

,Unnamed: 0,DATE,ID,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,...,YEAR_DAY,sin_WEEK_DAY,cos_WEEK_DAY,sin_YEAR_DAY,cos_YEAR_DAY,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,PRECIPITATIONS
